# LSTM con embeddings

Modelo básico con los embeddings de fasttext

In [1]:
%ls ../data/dev_es/

dev_es.tsv  train_es.tsv


In [2]:
import pandas as pd
df_train = pd.read_table("../data/dev_es/train_es.tsv", index_col="id")
df_dev = pd.read_table("../data/dev_es/dev_es.tsv", index_col="id")

text_train, y_train = df_train["text"], df_train["HS"]
text_dev, y_dev = df_dev["text"], df_dev["HS"]

Tengo que hacer dos cosas:

- Primero, convertir los tweets a secuencias de texto
- Luego, paddear las secuencias a cierta longitud (Keras necesita esto para poder paralelizar cálculo)

In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

num_words = 200000

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(text_train)

X_train = tokenizer.texts_to_sequences(text_train)
X_dev = tokenizer.texts_to_sequences(text_dev)

max_length = 30

X_train = pad_sequences(X_train, max_length)
X_dev = pad_sequences(X_dev, max_length)

Using TensorFlow backend.


Carguemos embeddings

In [4]:
import os
from glob import glob

path_to_embeddings = os.path.expanduser("/home/jmperez/WordVectors/")

print("Available embeddings: ", glob(os.path.join(path_to_embeddings, "*.vec")))

Available embeddings:  ['/home/jmperez/WordVectors/UBA_w5_200.vec', '/home/jmperez/WordVectors/wiki.es.vec', '/home/jmperez/WordVectors/UBA_w5_300.vec']


# Twitter Embeddings

In [5]:
import numpy as np

word_to_vec = {}

with open(os.path.join(path_to_embeddings, "UBA_w5_300.vec")) as f:
    for line in f:
        values = line.split()
        word = values[0]
        try:
            vec = np.asarray(values[1:], dtype="float32")
        except:
            print(("*" * 80  + "\n")*3)
            print("Problema con la sig línea:")
            print(values[:10])
            word = values[1]
            vec = np.asarray(values[2:], dtype="float32")
        word_to_vec[word] = vec
        
embedding_size = len(word_to_vec["hola"])

********************************************************************************
********************************************************************************
********************************************************************************

Problema con la sig línea:
['.', '.', '-0.22232', '0.0052569', '0.47066', '0.13836', '0.15991', '0.19504', '0.00067885', '0.020299']
********************************************************************************
********************************************************************************
********************************************************************************

Problema con la sig línea:
['.', '...', '-0.11666', '-0.083768', '0.028919', '0.29973', '0.21017', '0.27808', '0.063251', '0.090223']
********************************************************************************
********************************************************************************
********************************************************************************

P

In [6]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((num_words, embedding_size))
for word, i in tokenizer.word_index.items():
    embedding_vector = word_to_vec.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [7]:
embedding_matrix.shape

(200000, 300)

In [15]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout, Conv1D, Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence


model = Sequential()
model.add(Embedding(num_words, embedding_size, input_length=max_length, 
                    weights=[embedding_matrix], trainable=False))
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.5))
model.add(Conv1D(64, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [16]:
model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=10, batch_size=32)


Train on 4469 samples, validate on 500 samples
Epoch 1/10
4469/4469 [==============================] - 8s 2ms/step - loss: 0.6165 - acc: 0.6682 - val_loss: 0.5564 - val_acc: 0.7320
Epoch 2/10
4469/4469 [==============================] - 7s 2ms/step - loss: 0.5184 - acc: 0.7521 - val_loss: 0.5269 - val_acc: 0.7300
Epoch 3/10
4469/4469 [==============================] - 7s 2ms/step - loss: 0.4538 - acc: 0.7953 - val_loss: 0.4821 - val_acc: 0.7740
Epoch 4/10
4469/4469 [==============================] - 7s 2ms/step - loss: 0.4074 - acc: 0.8194 - val_loss: 0.4911 - val_acc: 0.7620
Epoch 5/10
4469/4469 [==============================] - 7s 2ms/step - loss: 0.3619 - acc: 0.8414 - val_loss: 0.4961 - val_acc: 0.8000
Epoch 6/10
4469/4469 [==============================] - 7s 2ms/step - loss: 0.3211 - acc: 0.8662 - val_loss: 0.4606 - val_acc: 0.7900
Epoch 7/10
4469/4469 [==============================] - 7s 2ms/step - loss: 0.2740 - acc: 0.8787 - val_loss: 0.4824 - val_acc: 0.8020
Epoch 8/10
4469

In [17]:
loss, accuracy = model.evaluate(X_dev, y_dev)

print("Loss function: {:.3f}".format(loss))
print("Accuracy: {:.2f}%".format(accuracy*100))

500/500 [==============================] - 0s 616us/step
Loss function: 0.596
Accuracy: 80.80%


## Bidirectional LSTM

In [18]:
from keras.layers import Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout, Conv1D, Flatten, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence


model = Sequential()
model.add(Embedding(num_words, embedding_size, input_length=max_length, 
                    weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Conv1D(64, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [19]:
model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=10, batch_size=32)


Train on 4469 samples, validate on 500 samples
Epoch 1/10
4469/4469 [==============================] - 14s 3ms/step - loss: 0.6127 - acc: 0.6729 - val_loss: 0.5643 - val_acc: 0.6920
Epoch 2/10
4469/4469 [==============================] - 12s 3ms/step - loss: 0.5030 - acc: 0.7568 - val_loss: 0.5316 - val_acc: 0.7280
Epoch 3/10
4469/4469 [==============================] - 12s 3ms/step - loss: 0.4516 - acc: 0.7892 - val_loss: 0.4571 - val_acc: 0.7840
Epoch 4/10
4469/4469 [==============================] - 12s 3ms/step - loss: 0.3918 - acc: 0.8230 - val_loss: 0.4489 - val_acc: 0.7840
Epoch 5/10
4469/4469 [==============================] - 12s 3ms/step - loss: 0.3326 - acc: 0.8575 - val_loss: 0.4987 - val_acc: 0.7920
Epoch 6/10
4469/4469 [==============================] - 12s 3ms/step - loss: 0.2838 - acc: 0.8823 - val_loss: 0.5279 - val_acc: 0.7740
Epoch 7/10
4469/4469 [==============================] - 12s 3ms/step - loss: 0.2556 - acc: 0.8962 - val_loss: 0.6069 - val_acc: 0.7660
Epoch 8/

In [20]:
loss, accuracy = model.evaluate(X_dev, y_dev)

print("Loss function: {:.3f}".format(loss))
print("Accuracy: {:.2f}%".format(accuracy*100))

500/500 [==============================] - 1s 1ms/step
Loss function: 0.776
Accuracy: 78.60%
